# CSCI E-7: Introduction to Programming with Python
## Graduate Student Project: Six Degrees of Wikipedia
### Kevin Friedman



### Introduction

In [1]:
from bs4 import BeautifulSoup
import requests

r = requests.get("https://en.wikipedia.org/wiki/Boston_Bruins")   
page = r.text

# create BeautifulSoup object from the html                    
soup = BeautifulSoup(page, "lxml")

# look within all <a> tags (all hyperlinks) in topic            
for link in soup.find_all('a'):
    print(link.get('href'))


None
/wiki/Wikipedia:Pending_changes
//en.wikipedia.org/w/index.php?title=Special:Log&type=review&page=Boston_Bruins
#mw-head
#p-search
/wiki/2016%E2%80%9317_Boston_Bruins_season
/wiki/File:Boston_Bruins.svg
/wiki/Eastern_Conference_(NHL)
/wiki/Atlantic_Division_(NHL)
/wiki/1924%E2%80%9325_NHL_season
/wiki/TD_Garden
/wiki/Boston
/wiki/File:ECA-Uniform-BOS.PNG
#cite_note-1
/wiki/New_England_Sports_Network
/wiki/WBZ-FM
/wiki/Delaware_North_Companies
/wiki/Jeremy_Jacobs
/wiki/Charlie_Jacobs
/wiki/Don_Sweeney
/wiki/Coach_(ice_hockey)
/wiki/Bruce_Cassidy
/wiki/Captain_(ice_hockey)
/wiki/Zdeno_Ch%C3%A1ra
/wiki/Providence_Bruins
/wiki/American_Hockey_League
/wiki/Atlanta_Gladiators
/wiki/ECHL
/wiki/Stanley_Cup
/wiki/1929_Stanley_Cup_Finals
/wiki/1939_Stanley_Cup_Finals
/wiki/1941_Stanley_Cup_Finals
/wiki/1970_Stanley_Cup_Finals
/wiki/1972_Stanley_Cup_Finals
/wiki/2011_Stanley_Cup_Finals
/wiki/1987%E2%80%9388_NHL_season
/wiki/1989%E2%80%9390_NHL_season
/wiki/2010%E2%80%9311_NHL_season
/wiki/20

### degrees()

Function "degrees" calculates number of degrees (degrees = # of links away) of separation on Wikipedia for two user inputted topics.  Degrees requires 0 parameters.  

It first takes the two inputted topic and finds the Wikipedia URL's for them.  Then, it finds all links on the Wikipedia page for the first topic.  

If the link to the second topic is on the page of the first topic, it returns the two topics and the number of degrees away (in this case it would be one degree).

If link to the second topic is not on the page of the first topic, it then searches through all of the pages linked to in the first topic.

If link to the second topic is on one of these pages, it returns the two topics and the number of degrees away (in this case it would be two degrees)

Repeat this process until 1) the second topic is found or 2) six degrees have been searched without finding the second topic 

For example, if a user inputs “Boston Bruins” and “Boston Red Sox”, the program would return 2 because the path of Wikipedia pages between them is  “Boston Bruins” → “Boston” → “Boston Red Sox”.  

In [1]:
from bs4 import BeautifulSoup
import requests

def degrees():
    
    # create empty list that will later store output to return
    output_list = []
    
    # while the user still wants to learn about how many degrees away topics are
    while True:

        t1 = input("\nPlease input the 1st topic: ")
        t2 = input("Please input the 2nd topic: ")

        degree = 1

        # VERY useful code from Alan's April 20th section to take first selection from
        # Google's "I'm Feeling Lucky" feature

        URL_TEMPLATE = "http://www.google.com/search?q=site:wikipedia.org+%s&btnI"

        # create response object t1r from url
        t1r= requests.get(URL_TEMPLATE % t1)

        #make topic 1 equal to the tail of the url
        # (so for t1=="Boston Red Sox" the topic1 value == "Boston_Red_Sox")
        topic1 = t1r.url[30:]

        # do same for topic 2
        t2r= requests.get(URL_TEMPLATE % t2)
        topic2 = t2r.url[30:]

        # create dictionary with keys intialized to integers 0-6
        link_dict = {i: set() for i in range(0,7)}

        # make degree 0 topic 1
        link_dict[0] = set([topic1])
        # loop through each degree
        while degree<=6:
            print("Checking degree {}...".format(degree))

            # initialize empty set to store links for each degree in
            # it will restart as empty set each iteration.
            # using set over list for efficiency so that I'm not 
            # storing anything more than once in each 

            temp_set = set()

            # loop through each topic in the previous degree
            for topic in link_dict[degree-1]:

                # create variable page containing all html code in url
                r = requests.get("https://en.wikipedia.org/wiki/" + topic)   
                page = r.text

                # create BeautifulSoup object from the html                    
                soup = BeautifulSoup(page, "lxml")

                # look within all <a> tags (all hyperlinks) in topic            
                for link in soup.find_all('a'):

                    # not every <a> tag works so I need to catch typeerror exceptions
                    # because not every <a> tag contains a url

                    try:

                        # create strings with head of url ("/wiki/")
                        # and tail of url (a new topic)

                        head = link.get('href')[0:6]
                        tail = link.get('href')[6:]


                        # make sure it doesn't already exist in dictionary
                        # to increase efficiency

                        exists_test = 0
                        for i in link_dict.keys():
                            if tail in link_dict[i]:
                                exists_test == 1
                                break

                        # if the url in <a> tag is another wikipedia page
                        # add it to our set for the current degree

                        if head=="/wiki/" and exists_test == 0:
                            temp_set.add(tail)

                    except TypeError:            
                        None

            # assign the set containing all wikipedia topics within the current
            # degree to the degree

            link_dict[degree] = temp_set

            # if topic2 is in topic 1, return degrees away

            if topic2 in link_dict[degree]:
                
                # append tuple with the two topics and degrees away
                output_list.append((t1, t2, degree))
                
                # print out number of degrees away (if degree is 1, print out 1 degree away)
                if degree == 1:
                    print ("\n{0} is 1 degree away from {2}.".format(t2, degree, t1))
                else:
                    print ("\n{0} is {1} degrees away from {2}.".format(t2, degree, t1))
                
                # ask if user wants to ask about more topics
                question = input("\nAre you curious about other topics (Yes / No)? ")
                question = question.lower()
                
                # if they do, break  send function back to "while True:" line
                # else return the list of tuples containing topics and # of degrees away

                if question != "yes":
                    return output_list
                
                else:
                    break

            # else, iterate through the next degree
            else:
                degree +=1
                #print (link_dict)

        # if we're more than six degrees in, let user know
        if degree > 6:
            print( "\nSorry, more than six degrees away!")
            
        
# cow, chicken
# Paper Diamond, Daft Punk


### degrees_bars()

Function "degrees_bars" takes the list containing the results of "degrees" function and creates a bar graph showing the number of Wikipedia degrees of separation between each pair of topics in the list.

In [2]:
from bokeh.charts import Bar, show, output_notebook
from bokeh.models import HoverTool, Range1d

def degrees_bars(degrees_output):
    
    output_notebook()

    # best support is with data in a format that is table-like

    data = {"Topics":[] , "Degrees":[]}
    
    # topics list will contain the 2 outputted topics from degrees function
    # degrees list will contain the number of degrees outputted from degrees function
    
    topics_list = []
    degrees_list = []
    for i in range(len(degrees_output)):    
        topics_list.append("{} to {}".format(degrees_output[i][0], degrees_output[i][1]))
        degrees_list.append(degrees_output[i][2])

    data["Topics"] = topics_list
    data["Degrees"] = degrees_list


    # create bar graph with results
    p = Bar(data, 
               values='Degrees',
               label='Topics',
               agg='sum',
               ylabel = "Number of Degrees",
               xlabel = "Topics",
               title="Number of Degrees Between Topics",
               legend=False,
               width=900)

    p.xaxis.major_label_orientation = "horizontal"
    p.axis.axis_label_text_font_size = "16pt"
    p.axis.axis_label_text_font = "Ariel"
    p.axis.axis_label_text_font_style = "normal"
    p.title.text_font_size = "18pt"
    p.title.text_font = "Ariel"
    p.title.text_font_style = "normal"
    p.title.align = "center"
    p.y_range = Range1d(0, 6)
    p.axis.major_label_text_font_size= "14pt"
    show(p)


### Examples

In [ ]:
print(degrees_bars(degrees()))

In [3]:
print(degrees_bars(degrees()))


Please input the 1st topic: Huntington Lambert
Please input the 2nd topic: Harvard University
Checking degree 1...

Harvard University is 1 degree away from Huntington Lambert.

Are you curious about other topics (Yes / No)? yes

Please input the 1st topic: Huntington Lambert
Please input the 2nd topic: World War II
Checking degree 1...
Checking degree 2...

World War II is 2 degrees away from Huntington Lambert.

Are you curious about other topics (Yes / No)? yes

Please input the 1st topic: Huntington Lambert
Please input the 2nd topic: total war
Checking degree 1...
Checking degree 2...
Checking degree 3...

total war is 3 degrees away from Huntington Lambert.

Are you curious about other topics (Yes / No)? no


Loading BokehJS ...

None
